In [ ]:
# First we load autoload, so we don't need to keep restarting the kernel to get
# new definitions. This ensures that functions are reloaded from the file whenever
# the file is changed.
%load_ext autoreload
%autoreload 2

In [12]:
# All the stuff we need
from lark import *
from lark.tree import Tree
from lark.lexer import Token
from ast import literal_eval
import numpy as np

from helpers import *
from show import *
from showcq import *
from type import *
from PE import *

In [67]:
#
# The following reads in a full program and parses it into an AST `program_tree`.
#
cqparse = Lark.open("CQ.lark",parser='lalr', start="program")
program = read_file("../CQ-programs/qft2.cq")

# Remove comment to test with `initialize2.cq`
#program = read_file("../CQ-programs/initialize2.cq") 
program_tree=cqparse.parse(program, start="program")
# If we've parsed the program properly, we can translate it back into CQ syntax
print(showcq_program(program_tree))


qft(qbit q[d]){
  int i = 0 ;
  while((i < (d - 1)))
    {
      int j = (i + 1) ;
      int angle = 2 ;
      H q[i] ;
      H q[(i + 1)] ;
      while((j < d))
        {
          Ry(((2 * pi) / angle)) q[i] if q[j]) ;
          angle = (angle * 2) ;
          j = (j + 1) ;
        }

      i = (i + 1) ;
    }

}
)


In [76]:
# It's easiest to test your program bit by bit. 
# We use the start symbol to get parsers for individual syntactical elements.
# The following are the parsers for expressions, declarations, and statements.
exp_parser  = Lark.open("CQ.lark", start="exp")
decl_parser = Lark.open("CQ.lark", start="declaration")
stat_parser = Lark.open("CQ.lark", start="statement")

# Some test examples in order of increasing difficulty
# Make sure that the variables you want to be statically evaluated
# are defined in your static_input dictionary below.
# We will use the ASTs to test the type checker and the PE further below.
# If you get an error already here, then there is an issue with your parser.
# Change these up to experiment.
t = exp_parser.parse("2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]))")
d = decl_parser.parse("float th;")
#d = decl_parser.parse("float th = 2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]));")
#d = decl_parser.parse("float x[4] = { 1/sqrt(2), 1/sqrt(4), 1/sqrt(6), 1/sqrt(12) };")
s = stat_parser.parse("a[0] = 2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]));")
#s = stat_parser.parse("if (2+2==4) a[0] = 1; else a[1] = 1;") 
#s = stat_parser.parse("if (2+2 < 5*a[1]) a[1] = 1; else a[0] = 1;")
#s = stat_parser.parse("{}")
#s = stat_parser.parse("while (i<2){ int j = a[i]; a[i] = a[3-i]; a[3-i] = j; i=i+1; }")
#s = stat_parser.parse("while (i<2){ int j = b[i]; b[i] = b[3-i]; b[3-i] = j; i=i+1; }")
#s = stat_parser.parse("{ a[i] = i+1; i = a[i]; }")
#s = stat_parser.parse("{ int i; int j=1; i = 0; while (j < 10) { a[i+1] = 0; i = i + 1; } }")
initialize_stat="""{
    float a[4] = {1/sqrt(2), 1/sqrt(4), 1/sqrt(6), 1/sqrt(12)};
    qbit  q[2];
    float th1 = 2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]));
    float th2 = 2*arctan2(a[3],a[1]); 
    float th3 = 2*arctan2(a[2],a[0]);

    Ry(th1) q[0];
    Ry(th2) q[1] if q[0];
    not q[0];
    Ry(th3) q[1] if q[0];
    not q[0];
}
"""
#s = stat_parser.parse(initialize_stat)

In [79]:
# Test of the type checker on the exp, declaration, and statement ASTs.
type_env0=[{},{'a': "float[4]", 'b':'int[4]', "i": "int"}]

# Test type checker on smaller syntactical elements:
print(type_exp(t,type_env0))
print(type_declaration(d,type_env0))
#print(showcq_declaration(d))
#print(showcq_statement(s))
print(type_statement(s,type_env0))


float
('th', 'float')
True


In [80]:
# Type check the entire program
type_program(program_tree)

Procedure qft is well-typed


True

In [89]:
type_env0=[{},{'a': "float[4]"}]
static_input = {'a': np.sqrt([1/2.,1/4.,1/6., 1/12.]),'d':4}
value_env0=[static_input]
#type_env0=[{},{'a': "int[4]", 'i': "int"}]
#value_env0=[{'a': np.array([4,3,2,1]), 'i':0}]


#Test PE for a declaration AST:
(ped,static) = PE_declaration(d,value_env0)
if not static:
    print(showcq_declaration(ped))
else: 
    print(f"`{showcq_declaration(d)}` fully evaluated, environment is:\n {value_env0}")


`float th ;` fully evaluated, environment is:
 [{'a': array([0.70710678, 0.5       , 0.40824829, 0.28867513]), 'd': 4, Token(Token('TERMINAL', 'ID'), 'th'): 0}]


In [90]:

# Test PE for a statement AST:
(pes,static) = PE_statement(s,value_env0)
if not static:
    #print(f"residual statement:\n\t{pes}\noriginal statement:\n\t{s}")
    #print(pes.pretty())
    print(showcq_statement(pes))
else: 
    print(f"{showcq_statement(s)} fully evaluated,\n environment is {value_env0}")    

a[0] = (2 * arccos(sqrt(((a[0] * a[0]) + (a[2] * a[2]))))) ; fully evaluated,
 environment is [{'a': array([1.23095942, 0.5       , 0.40824829, 0.28867513]), 'd': 4, Token(Token('TERMINAL', 'ID'), 'th'): 0}]


In [91]:
# Test PE for a full program (Once you get this far!):
pt_res = PE_program(pt,static_input) 

In [92]:
print(showcq_program(pt_res))

qft(qbit q[4]){
  {
    H q[0] ;
    H q[1] ;
    {
      Ry(3.141592653589793) q[0] if q[1]) ;
      Ry(1.5707963267948966) q[0] if q[2]) ;
      Ry(0.7853981633974483) q[0] if q[3]) ;
    }

  }

  {
    H q[1] ;
    H q[2] ;
    {
      Ry(3.141592653589793) q[1] if q[2]) ;
      Ry(1.5707963267948966) q[1] if q[3]) ;
    }

  }

  {
    H q[2] ;
    H q[3] ;
    Ry(3.141592653589793) q[2] if q[3]) ;
  }

}
)
